In [ ]:
pip install gradio jupyter ipywidgets ollama

In [1]:
import ollama
ollama.pull('mistral')

{'status': 'success'}

In [ ]:
from ollama import Client
client = Client(host='http://localhost:11434')
response = client.chat(model='mistral', messages=[
  {
    'role': 'user',
    'content': 'Why is the sky blue?',
  },
])
print(response['message']['content'])

In [ ]:
import ollama

stream = ollama.chat(
    model='mistral',
    messages=[{'role': 'user', 'content': 'Why is the sky blue?'}],
    stream=True,
)

for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)

In [ ]:
ollama.list()

In [ ]:
ollama.show('mistral:7b')

In [2]:
import gradio as gr
from ollama import Client

def mistralChat(text):
    
    client = Client(host='http://localhost:11434')
    response = client.chat(model='mistral', messages=[
    {
        'role': 'user',
        'content': text,
    },
    ])
    return response['message']['content']

In [7]:
ollamaList = ["mistral"]
with gr.Blocks() as server:
    with gr.Dropdown(choices=ollamaList, value="mistral", type="value",):
    
        model_input = gr.Textbox(label="Your Question:", 
                                value="What’s your question?", interactive=True)
        ask_button = gr.Button("Ask")
        model_output = gr.Textbox(label="The Answer:", interactive=False, 
                                value="Answer goes here...")
    
    ask_button.click(mistralChat, inputs=[model_input], outputs=[model_output])

server.launch(share=True)

AttributeError: __enter__